In [ ]:
import threading
import cv2
import os
import csv
from datetime import datetime
from deepface import DeepFace
import pyttsx3

cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

counter = 0
match_info = None
match_info_lock = threading.Lock()

reference_folder = "images"
reference_images = {}

for file_name in os.listdir(reference_folder):
    if file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        person_name = os.path.splitext(file_name)[0]
        image_path = os.path.join(reference_folder, file_name)
        reference_images[person_name] = cv2.imread(image_path)

DeepFace.build_model("Facenet")

csv_file_path = r"C:\Users\sures\Downloads\project\daily_record.csv"

def log_to_csv(person_name):
    if person_name:
        now = datetime.now()
        date = now.strftime("%Y-%m-%d")
        time = now.strftime("%H:%M:%S")
        with open(csv_file_path, mode='a', newline='') as file:
            writer = csv.writer(file)
            if os.stat(csv_file_path).st_size == 0:
                writer.writerow(["Date", "Time", "NAME"])
            writer.writerow([date, time, person_name])

def check_face(frame):
    global match_info
    try:
        with match_info_lock:
            for person, reference_img in reference_images.items():
                if reference_img is not None:
                    result = DeepFace.verify(frame, reference_img.copy(), model_name="Facenet")
                    if result['verified']:
                        match_info = person
                        speak_text("you successfull logedin")
                        log_to_csv(match_info)
                        break
            else:
                match_info = None
                speak_text("Not Verified")
    except Exception as e:
        print("Error verifying face:", e)

def speak_text(text):
    engine = pyttsx3.init()
    engine.setProperty('rate', 150)   
    engine.setProperty('volume', 0.9)
    engine.say(text)
    engine.runAndWait()

while True:
    ret, frame = cap.read()

    if ret:
        if counter % 30 == 0:
            try:
                threading.Thread(target=check_face, args=(frame.copy(),)).start()
            except Exception as e:
                print("Error starting thread:", e)
        counter += 1

        with match_info_lock:
            if match_info:
                cv2.putText(frame, f"Verified: {match_info}", (20, 450), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            else:
                cv2.putText(frame, " Not Verified", (20, 450), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        cv2.imshow("video", frame)

    key = cv2.waitKey(1)

    if key == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
